In [3]:
import pandas as pd
import numpy
import numpy as np
from os.path import exists
from os import mkdir
from os import listdir
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import matplotlib as mpl
import matplotlib
import matplotlib.pyplot as plt
#from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib import cm
import matplotlib.cm as cm
from mpl_toolkits import mplot3d
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.axes_grid1.inset_locator import InsetPosition
import matplotlib.colors as cols
import matplotlib.gridspec as grid_spec
import scipy
import seaborn as sns
import re
import csv as csv
from itertools import cycle
from sklearn.neighbors import KernelDensity
import trimesh
import sys
from numpy.random import randn
from scipy import array, newaxis
from scipy.spatial import Delaunay
import proplot as pplt
from matplotlib.gridspec import GridSpec
from matplotlib.offsetbox import AnchoredText
import open3d as o3d
import os
import sys
import copy
import trimesh
import open3d as o3d
#%env OPEN3D_CPU_RENDERING true

#install.packages("viridis")
#library(viridis)


mpl.rcParams["figure.dpi"] = 300
mpl.rcParams["legend.fontsize"] = 12
mpl.rcParams["font.size"] = 12

# import matplotlib.tri as mtri

# from scipy.interpolate import griddata

PLOTDIR = "plots"
DATADIR = "data"

DIRS = [PLOTDIR, DATADIR]

def plot_savepath(filename):
    return PLOTDIR + f"/{filename}"

def data_path(filename):
    return DATADIR + f"/{filename}"

for d in DIRS:
    if not exists(d):
        mkdir(d)
        
        
output_dir = f"figures/experiments/"
# Path(output_dir).mkdir(parents=True, exist_ok=True)

#data files are shown as AICS-31 1857 Dots (1)

In [7]:
#"Organoid_Names" in this case are the file names of the positional data from the organoid staining#

Organoid_Names = ['HW8D60SUPB15_VOL_1.xls',
                  'HW8D60SUPB15_VOL_2.xls',
                  'HW8D60SUPB15_VOL_3.xls', 
                  'HW8D60697_VOL_1.xls',
                  'HW8D60697_VOL_2.xls',
                  'HW8D60697_VOL_3.xls',
                  'HW8D60KASUMI2_VOL_1.xls',
                  'HW8D60KASUMI2_VOL_2.xls', 
                  'HW8D60KASUMI2_VOL_3.xls']

organoid_pts = []

for name in Organoid_Names:
    df = pd.read_excel(data_path(name), header = None, skiprows =1)
    df = df.rename(columns = df.iloc[0]).drop(df.index[0])
    df_drop = df.drop(columns=['Unit', 'Category', 'Collection', 'Time', 'ID'])
    pts = df_drop.to_numpy()
    organoid_pts.append(pts)

FileNotFoundError: [Errno 2] No such file or directory: 'data/HW8D60697_VOL_1.xls'

In [6]:
Entity_Names = ['HW8D60SUPB15_DOTS_1.xls',
                'HW8D60SUPB15_DOTS_2.xls',
                'HW8D60SUPB15_DOTS_3.xls', 
                'HW8D60697_DOTS_1.xls',
                'HW8D60697_DOTS_2.xls',
                'HW8D60697_DOTS_3.xls',
                'HW8D60KASUMI2_DOTS_1.xls',
                'HW8D60KASUMI2_DOTS_2.xls', 
                'HW8D60KASUMI2_DOTS_3.xls']

entity_pts = []

for name in Entity_Names:
    df = pd.read_excel(data_path(name), header = None, skiprows =1)
    df = df.rename(columns = df.iloc[0]).drop(df.index[0])
    df_drop = df.drop(columns=['Unit', 'Category', 'Collection', 'Time', 'ID'])
    pts = df_drop.to_numpy()
    entity_pts.append(pts)

FileNotFoundError: [Errno 2] No such file or directory: 'data/HW8D60697_DOTS_1.xls'

In [ ]:
assert len(organoid_pts)==len(entity_pts)

distances = []

for index in range(len(organoid_pts)):
    # Organoids
    pcd_org = o3d.geometry.PointCloud()
    pcd_org.points = o3d.utility.Vector3dVector(organoid_pts[index])
    pcd_org.compute_convex_hull()
    pcd_org.estimate_normals()
    pcd_org.orient_normals_consistent_tangent_plane(10)
    
    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        mesh_org, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
            pcd_org, depth=10, scale=10, linear_fit=True)
    # print(mesh)
    mesh_org.paint_uniform_color([0.8, 0, 0])
    mesh_org.compute_vertex_normals()
    mesh_org.filter_smooth_simple(number_of_iterations=1)
    
    mesh_to_fill_org = o3d.t.geometry.TriangleMesh.from_legacy(mesh_org)
    mesh_to_fill_org.fill_holes(hole_size=10)
    final_mesh_org = mesh_to_fill_org.to_legacy()
    
    # Entities
    pcd_ent = o3d.geometry.PointCloud()
    pcd_ent.points = o3d.utility.Vector3dVector(entity_pts[index])
    pcd_ent.compute_convex_hull()
    pcd_ent.estimate_normals()
    pcd_ent.orient_normals_consistent_tangent_plane(10)
    
    with o3d.utility.VerbosityContextManager(
            o3d.utility.VerbosityLevel.Debug) as cm:
        mesh_ent, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
            pcd_ent, depth=10, scale=10, linear_fit=True)
    # print(mesh)
    mesh_ent.paint_uniform_color([0.8, 0, 0])
    mesh_ent.compute_vertex_normals()
    mesh_ent.filter_smooth_simple(number_of_iterations=1)
    
    mesh_to_fill_ent = o3d.t.geometry.TriangleMesh.from_legacy(mesh_ent)
    mesh_to_fill_ent.fill_holes(hole_size=10)
    final_mesh_ent = mesh_to_fill_ent.to_legacy()
    
    # Calculation of distances
    mesh_org_toleg = o3d.t.geometry.TriangleMesh.from_legacy(final_mesh_org)
    mesh_ent_toleg = o3d.t.geometry.TriangleMesh.from_legacy(final_mesh_ent)
    
    scene = o3d.t.geometry.RaycastingScene()
    mesh_ids = {}
    mesh_ids[scene.add_triangles(mesh_org_toleg)] = 'surface'
    mesh_ids[scene.add_triangles(mesh_ent_toleg)] = 'cells'

    query_point = np.asarray(entity_pts[index]).astype('float32')

    unsigned_distance = scene.compute_distance(query_point)
    distances.append(unsigned_distance.numpy())
    
    

In [ ]:
dict_keys = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']

d = dict(zip(dict_keys, distances))

In [ ]:
df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in d.items() ]))

In [ ]:
df = df.melt(var_name = "Entity", value_name = "Distance to Surface (\u03BCm)")

df.loc[df['Entity'] == 'A', 'Types'] = 'Repeat 1'  
df.loc[df['Entity'] == 'B', 'Types'] = 'Repeat 2'  
df.loc[df['Entity'] == 'C', 'Types'] = 'Repeat 3'  
df.loc[df['Entity'] == 'D', 'Types'] = 'Repeat 1'  
df.loc[df['Entity'] == 'E', 'Types'] = 'Repeat 2'  
df.loc[df['Entity'] == 'F', 'Types'] = 'Repeat 3'  
df.loc[df['Entity'] == 'G', 'Types'] = 'Repeat 1'  
df.loc[df['Entity'] == 'H', 'Types'] = 'Repeat 2'  
df.loc[df['Entity'] == 'I', 'Types'] = 'Repeat 3'

In [ ]:
df["Entity"] = df["Entity"].replace({'A':'SUPB15',
              'B':'SUPB15',
              'C':'SUPB15',
              'D':'697',
              'E':'697',
              'F':'697',
              'G':'Kasumi2',
              'H':'Kasumi2',
              'I':'Kasumi2'}) 

In [ ]:
df[np.isfinite(df["Distance to Surface (\u03BCm)"])]
df_final = df[df["Distance to Surface (\u03BCm)"] > 0.0001]

In [ ]:
df_final

In [ ]:
#leukemia data
#df.rename(columns={'A': 'SUPB15_1',
#                   'B': 'SUPB15_2',
#                   'C': 'SUPB15_3',
#                   'D': '697_1',
#                   'E': '697_2',
#                   'F': '697_3',
#                   'G': 'Kasumi2_1',
#                   'H': 'Kasumi2_2',
#                   'I': 'Kasumi2_3'}, inplace=True)

#df.to_excel("unmelted_otherleukemia_data.xlsx", sheet_name='unmeltedLEUK') 
#df_leukemia = df.melt(var_name = "Entity", value_name = "Distance to Surface (\u03BCm)")
#df_leukemia.to_excel("melted_otherleukemia_data.xlsx", sheet_name='meltedLEUK') 
#df_leukemia['Location'] = ['below-surface' if x >= 10 else 'near-surface' for x in df_leukemia["Distance to Surface (\u03BCm)"]]

In [ ]:
#df_leukemia.loc[df_leukemia['Entity'] == 'SUPB15_1', 'Types'] = 'Repeat 1'  
#df_leukemia.loc[df_leukemia['Entity'] == 'SUPB15_2', 'Types'] = 'Repeat 2'  
#df_leukemia.loc[df_leukemia['Entity'] == 'SUPB15_3', 'Types'] = 'Repeat 3'  
#df_leukemia.loc[df_leukemia['Entity'] == '697_1', 'Types'] = 'Repeat 1'  
#df_leukemia.loc[df_leukemia['Entity'] == '697_2', 'Types'] = 'Repeat 2'  
#df_leukemia.loc[df_leukemia['Entity'] == '697_3', 'Types'] = 'Repeat 3'  
#df_leukemia.loc[df_leukemia['Entity'] == 'Kasumi2_1', 'Types'] = 'Repeat 1'  
#df_leukemia.loc[df_leukemia['Entity'] == 'Kasumi2_2', 'Types'] = 'Repeat 2'  
#df_leukemia.loc[df_leukemia['Entity'] == 'Kasumi2_3', 'Types'] = 'Repeat 3'  

#df_leukemia[np.isfinite(df_leukemia["Distance to Surface (\u03BCm)"])]

In [ ]:
#for index in range(len(organoid_pts)):
#    vis = o3d.visualization.Visualizer()
#    vis.create_window(visible=False) #works for me with False, on some systems needs to be true
#    vis.add_geometry(final_mesh_org)
#    vis.update_geometry(final_mesh_org)
#    vis.poll_events()
#    vis.update_renderer()
#    vis.capture_screen_image('ORG_Mesh.png', do_render=True)
#    vis.destroy_window()
    
#for index in range(len(entity_pts)):
#    vis = o3d.visualization.Visualizer()
#    vis.create_window(visible=False) #works for me with False, on some systems needs to be true
#    vis.add_geometry(pcd_ent)
##    vis.update_geometry(pcd_ent)
#    vis.poll_events()
#    vis.update_renderer()
#    vis.capture_screen_image('EntPCD.png', do_render=True)
#    vis.destroy_window()

In [ ]:
# Drop rows with None/NaN
#df_final = df_pos[df_pos.notnull()]

In [ ]:
sns.set_theme(style="ticks")

fig, axes = plt.subplots(1, 2, figsize=(25, 10), gridspec_kw={'width_ratios': [1, 1]})
g = sns.stripplot(ax=axes[0], data=df_final,
              x="Entity", y="Distance to Surface (\u03BCm)",
              hue="Types", palette="GnBu", linewidth=0.5,
              size=5, alpha=0.3, jitter=.30, dodge=True,
                 )
g.set_yscale("log")
axes[0].grid(color='black', alpha=0.5, linestyle='dashed', linewidth=0.5)
axes[0].set_xlabel('Entity', fontweight ='bold')
axes[0].set_ylabel('Distance to Surface (\u03BCm)', fontweight ='bold')

# the non-logarithmic labels you want
yticks = [0.01, 0.1, 1, 10, 100, 1000]
g.set_yticks(yticks)
g.set_yticklabels(yticks)

g.tick_params(axis='x', rotation=45)
g.xaxis.label.set_visible(False)

sns.move_legend(
    g, loc="best", ncol=1, frameon=False, columnspacing=5, handletextpad=0
)

g.axhline(10, color='r')

g = sns.histplot(
    df_final,
    x="Distance to Surface (\u03BCm)", hue="Entity",
    bins=200,
    multiple="stack",
    palette="GnBu",
    element="bars",
    edgecolor=".1",
    common_norm=False,
    kde=False,
    pmax=1000,
    linewidth=.1,
    stat="count",
    log_scale=True,
)

sns.move_legend(
    g, loc="best", ncol=1, frameon=False, columnspacing=5, handletextpad=0)


axes[1].grid(color='black', alpha=0.5, linestyle='dashed', linewidth=0.5)
axes[1].set_xlabel('Distance to Surface (\u03BCm)', fontweight ='bold')
axes[1].set_ylabel('Count', fontweight ='bold')
#ticks = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
#g.set_yticks(ticks)
#g.set_yticklabels(ticks)

g.axvline(x=10, color='r')


#axes[1].set_title("Violinplot", fontsize=20)

#g = ax=axes[2].imshow(img_org)
#axes[2].set_xlabel('Position X (\u03BCm)', fontweight ='bold')
#axes[2].set_ylabel('Position Y (\u03BCm)', fontweight ='bold')

#g = ax=axes[3].imshow(img_ent)
#axes[3].set_xlabel('Position X (\u03BCm)', fontweight ='bold')
#axes[3].set_ylabel('Position Y (\u03BCm)', fontweight ='bold')


texts = ['S', 'T']
ax = fig.get_axes()
for a,l in zip(ax, texts):
    a.annotate(l, xy=(-0.1, 1.1), xycoords="axes fraction", fontsize=45, weight = 'bold')
#plt.suptitle('Label_Distance Consistent', fontsize = 20)

plt.savefig(plot_savepath('Seaborn_DepthAnalysis_other_entities.pdf'), bbox_inches='tight')
plt.savefig(plot_savepath('Seaborn_DepthAnalysis_other_entities.png'), dpi=300, bbox_inches='tight')